In [1]:
import os
from Dataset import Dataset
from DataLoader import DataLoader
from Speech import Speech
from preprocessors import *
from utils import scrape_speeches
from tqdm import tqdm
import json

/home/m_hovo/Desktop/KU Leuven/Modern Data Analytics/mda-project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



In [2]:
usr_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
cwd = os.getcwd()
path = f"{cwd}/dataset_typical"

In [3]:
if not os.path.exists(path) or len(os.listdir(path)) == 0:
    scrape_speeches("https://www.americanrhetoric.com/", path, usr_agent)

In [4]:
preprocessors = [
    RemoveExtras(names=True),
    # RemoveStopwords(),
    # Lemmatize(),
    # RemovePunctuation(),
    # Lower()
]
dataset = Dataset(path, preprocessors)
dataloader = DataLoader(dataset)

In [5]:
results_dir = f"{cwd}/results"
entities_saving_dir = f"{cwd}/results/entities_typical.csv"
entities = {
    "entities": [],
    "count": [],
    "proportion_in_speech": []
}

In [6]:
with tqdm(total=len(dataloader.dataset)) as progress_bar:
    for i, speech_content in enumerate(dataloader):
        speech = Speech(speech_content)
        speech_entities = speech.get_entities(labels=False)
        entities["entities"].append(speech_entities)
        entities["count"].append(len(speech_entities))
        entities["proportion_in_speech"].append(len(speech_entities) / speech.get_length())
        progress_bar.update(1)

100%|███████████████████████████████████████████| 77/77 [08:11<00:00,  6.38s/it]


In [7]:
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
with open(entities_saving_dir, "w") as write_file:
    json.dump(entities, write_file, indent=4)